# Constraint-based sampling for metabolic flux analysis (MFA)

Here we jump into the (for now) final step in our metabolic flux analysis (MFA) pipeline. After constructing an INCA script, running it in MATLAB and reimporting the data we're now here. This example notebook will guide you through different ways to integrate your MFA results into COBRA models and how to make them more reliable.

In [1]:
import pandas as pd
import escher
import cobra
from tabulate import tabulate
from gurobipy import Model as GRBModel
import re
from BFAIR.INCA import INCA_reimport
from BFAIR.INCA.sampling import (
    add_constraints,
    find_biomass_reaction,
    get_min_solution_val,
    replace_biomass_rxn_name,
    add_feasible_constraints,
    reshape_fluxes_escher,
    bound_relaxation,
)

Academic license - for non-commercial use only - expires 2021-05-28
Using license file /Users/matmat/gurobi.lic


#### INCA re-import

First, let's reimport the data using our `BFAIR INCA_reimport` tools

The data is taken from a different folder

In [2]:
%cd /Users/matmat/Documents/Random data/Stefano
filename = 'data/MFA_modelInputsData/TestFile.mat'

/Users/matmat/Documents/Random data/Stefano


Then we move back to our example folder to get the remaining input

In [3]:
%cd /Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples
#filename = 'data/MFA_modelInputsData/TestFile.mat'
simulation_info = pd.read_csv('data/MFA_modelInputsData/Re-import/experimentalMS_data_I.csv')
simulation_id = 'WTEColi_113C80_U13C20_01'

/Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples


Here we re-import the INCA output

In [4]:
reimport_data = INCA_reimport()
(fittedData,
 fittedFluxes,
 fittedFragments,
 fittedMeasuredFluxes,
 fittedMeasuredFragments,
 fittedMeasuredFluxResiduals,
 fittedMeasuredFragmentResiduals,
 simulationParameters) = reimport_data.reimport(
    filename,
    simulation_info,
    simulation_id
)

Here we import the model

In [5]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [6]:
original_solution = model.optimize()
original_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,-0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,-8.673617e-18
RNTR2c2,0.026541,-8.673617e-18


Let's see what happens now when we add our new bound constraints

In [7]:
model = add_constraints(model, fittedFluxes)
model.optimize()

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpgevl969f.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpw22d388u.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


<Solution infeasible at 0x7fed7a864a60>

Oh... That's not good... Well that sucks, seems like we have to deal with an infeasible solution. There are two straight forward ways: exclusion and relaxation

#### Dealing with infeasible solutions - exclusion

The easier way to deal with this issue is to simply exclude the constraints that render a model infeasible. We can do that by adding the calculated bounds one by one. If we come across a reaction whose bounds cause trouble, we restart the process and skip this one. This might have to be done a few times to exclude all troublemakers. the `add_feasible_constraints()` functions takes care of that for us. Let's reset the model first.

In [8]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [9]:
find_biomass_reaction(model)

['BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M']

In [10]:
min_val = get_min_solution_val(fittedFluxes, biomass_string='Biomass')

In [11]:
fittedFluxes = replace_biomass_rxn_name(fittedFluxes, biomass_string='Biomass', biomass_rxn_name='BIOMASS_Ec_iJO1366_core_53p95M')

In [12]:
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpvhuz3nzd.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpjhy4u1sw.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpj_to4qao.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp7afg659l.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding DAPDC
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpq43ov0yg.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpw1qn_l

The `model` is our newly constrained model and the problematic reactions can be listed in `problems`.

In [13]:
model

Name,iJO1366
Memory address,0x07fed5d7bb8e0
Number of metabolites,1805
Number of reactions,2583
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [14]:
problems

['ASPTA', 'DAPDC', 'BIOMASS_Ec_iJO1366_core_53p95M', 'GLNS', 'RPI']

Now let's see what an effect these new bounds had on the predicted growth rate (the objective value) of our model

In [15]:
new_bounds_solution = model.optimize()
new_bounds_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.146363e-01
EX_co2_e,22.825015,0.000000e+00
EX_cobalt2_e,-0.000019,-0.000000e+00
DM_4crsol_c,0.000166,0.000000e+00
...,...,...
RNDR4,0.000000,-1.275067e-02
RNDR4b,0.000000,-1.275067e-02
RNTR1c2,0.019444,1.387779e-17
RNTR2c2,0.020076,1.387779e-17


And here's the star of the show, our sampling method. We trust our models because... we have to! And because smart people that knew what they were doing set them up. So in order to gain more confidence in our MFA data, we sample the model after adding the calculated bound for some of the reactions and re-calculate the fluxes a number of time. Then, we take the mean and take that as the most trustworthy calculated flux. These fluxes can be visualized, for example in tools like `Escher`

In [16]:
sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp91p8xkbs.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp8kwvw7xn.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [17]:
sampled_fluxes

,EX_cm_e,EX_cmp_e,EX_co2_e,EX_cobalt2_e,DM_4crsol_c,DM_5drib_c,DM_aacald_c,DM_amob_c,DM_mththf_c,EX_colipa_e,...,UREAtex,RNDR2b,UREAtpp,RNDR3,RNDR3b,RNDR4,RNDR4b,RNTR1c2,RNTR2c2,RNTR3c2
0,0.0,0.0,25.944055,-3.560853e-11,3.254380e-10,1.443895e-08,0.0,2.918727e-12,3.659048e-07,9.349902e-09,...,-1.328251e-07,5.482530e-08,-1.328251e-07,8.937139e-08,4.003173e-09,9.836762e-08,2.705572e-08,1.867861e-07,0.224544,1.377517e-07
1,0.0,0.0,25.943928,-3.397009e-11,3.102143e-10,1.915771e-08,0.0,2.782191e-12,5.363311e-07,1.117056e-08,...,-4.369347e-07,5.482341e-08,-4.369347e-07,4.487528e-07,5.641417e-09,2.302766e-07,2.705479e-08,4.527207e-07,0.224510,1.359995e-07
2,0.0,0.0,25.942634,-1.478408e-09,1.368831e-08,3.296345e-08,0.0,1.227651e-10,2.495176e-07,5.393960e-07,...,-4.257927e-07,5.472849e-08,-4.257927e-07,4.485205e-07,5.594586e-09,2.300316e-07,2.700795e-08,4.012852e-06,0.224516,1.678929e-06
3,0.0,0.0,25.940084,-1.517500e-09,1.406920e-08,3.335619e-08,0.0,1.261812e-10,2.818958e-07,5.393530e-07,...,-4.636156e-07,5.467835e-08,-4.636156e-07,4.422681e-07,5.571231e-09,2.239797e-07,2.698321e-08,4.209111e-06,0.224501,1.726060e-06
4,0.0,0.0,26.286107,-1.461648e-09,1.357370e-08,1.568281e-07,0.0,1.217372e-10,1.143854e-06,6.112550e-07,...,-5.431293e-07,6.356535e-08,-5.431293e-07,4.574917e-07,9.956454e-09,2.582346e-07,3.136885e-08,2.312947e-05,0.161373,2.113256e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,31.700295,-1.426824e-07,1.321217e-06,6.334775e-04,0.0,1.184948e-08,8.403271e-03,1.125688e-04,...,-4.202608e-03,3.797031e-04,-4.202608e-03,1.313928e-02,5.208701e-03,1.587621e-02,3.003339e-02,1.119337e-02,0.002315,2.445162e-02
96,0.0,0.0,30.306040,-1.782349e-07,1.650347e-06,1.042769e-03,0.0,1.480132e-08,1.258445e-02,9.860148e-05,...,-4.705299e-03,1.111018e-03,-4.705299e-03,1.294436e-02,5.952318e-03,1.386831e-02,3.606162e-02,1.785853e-02,0.004431,2.848889e-02
97,0.0,0.0,30.569118,-1.712127e-07,1.590675e-06,1.039596e-03,0.0,1.426614e-08,1.274352e-02,3.943703e-05,...,-4.685588e-03,1.105731e-03,-4.685588e-03,1.297675e-02,5.946799e-03,1.390474e-02,3.601724e-02,1.678960e-02,0.004358,3.094420e-02
98,0.0,0.0,30.380746,-1.477088e-07,1.369220e-06,1.039274e-03,0.0,1.228000e-08,1.274117e-02,3.347506e-05,...,-4.624462e-03,1.139345e-03,-4.624462e-03,1.297601e-02,5.945052e-03,1.390265e-02,3.601523e-02,1.563159e-02,0.003514,3.091911e-02


In [18]:
fluxes_sampling = reshape_fluxes_escher(sampled_fluxes)

In [19]:
sampled_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = fluxes_sampling).display_in_notebook()
sampled_flux_map

There are also other ways to cosolidate the MFA calculations and the constraint based flux predictions. One of these is lMOMA (linear Minimization Of Metabolic Adjustment). MOMA assumes that the fluxes before and after adding the new constraints should be similar, so it aims to predicting an optimum for the newly constrined model while keeping the differences to the original model small. We suggest using pFBA (parsimonious Flux Balance Analysis) instead of regular FBA for this step as pFBA aims to keep the overal fluxes low.

In [20]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpnt_tsz71.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpnaxyqw_3.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpdrswpzvk.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpm8x1h48g.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding DAPDC
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp1ugeodfd.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpp5yta2cc.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding BIOMASS_Ec_iJO1366_core_53p95M
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp1_0xmihk.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpin_r2uyn.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding GLNS
Read LP format model from file /var/folders

In [21]:
pfba_solution = cobra.flux_analysis.pfba(model)

In [22]:
moma_after_MFA = cobra.flux_analysis.moma(
    model=model, solution=pfba_solution, linear=True)

In [23]:
moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745775968

In [24]:
room_after_MFA = cobra.flux_analysis.room(
    model=model, solution=pfba_solution, linear=True, delta=0.03, epsilon=0.001)

In [25]:
room_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745776016

In [26]:
results = [['Before adding new constraints:', round(original_solution.objective_value, 2)],
           ['New constraints relaxed FBA:', round(new_bounds_solution.objective_value, 2)],
           ['New constraints relaxed pFBA:', round(pfba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed MOMA:', round(moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed ROOM:', round(room_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)]]
print(tabulate(results, headers=["Method", "Biomass function"]))

Method                            Biomass function
------------------------------  ------------------
Before adding new constraints:                0.98
New constraints relaxed FBA:                  0.74
New constraints relaxed pFBA:                 0.74
New constraints relaxed MOMA:                 0.74
New constraints relaxed ROOM:                 0.74


Of course also the MOMA results can be visualized with `Escher`. The `reshape_fluxes_escher()` can take both pandas DataFrames or cobra solutions as an input.

In [27]:
fluxes_moma = reshape_fluxes_escher(moma_after_MFA)
moma_flux_map = escher.Builder('e_coli_core.Core metabolism',
                               reaction_data = fluxes_moma).display_in_notebook()
moma_flux_map

In [28]:
fluxes_room = reshape_fluxes_escher(room_after_MFA)
room_flux_map = escher.Builder('e_coli_core.Core metabolism',
                               reaction_data = fluxes_room).display_in_notebook()
room_flux_map

#### Dealing with infeasible solutions - relaxation

Another way of dealing with infeasible models is to relax the added constraints to the point that it works again. You will need to have the Gurobi solver installed for this. The same principle is used in the `BFAIR thermo` tools. For that we add our constraints to a model that will now be infeasible. Have I meantioned that this is much more elegant, better and that you should do that? It is. The other method is *fine* but you exclude reactions and, in general, it is always better to use as much as possible of the information that is available to you.

In [29]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [30]:
model = add_constraints(model, fittedFluxes)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpzgdfn195.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpbc1jqydt.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


In [31]:
model.optimize()

<Solution infeasible at 0x7fed61720970>

Then we make use of the handy `bound_relaxation()` function that will test our model to figure out which of these added bounds need to be adjusted and return a DataFrame that describes the affected functions and the gravity of the suggested changes. If we allow this function to be `desctructive` it will adjust the input model right away.

In [32]:
bound_relaxation(model, fittedFluxes, destructive=True, fluxes_to_ignore=['BIOMASS_Ec_iJO1366_core_53p95M'])

--- start ---
bound_relaxation takes 0h: 0min: 0sec to run
--- end ---


,lb_change,ub_change,subsystem
reaction,,,
ASPTA,-3.349375,0.000000,Alanine and Aspartate Metabolism
DAPDC_reverse_d3ab8,-0.040213,0.000000,Threonine and Lysine Metabolism
GLNS_reverse_59581,-0.767431,0.000000,Glutamate Metabolism
MTHFC_reverse_f6fcc,-0.515687,0.000000,Folate Metabolism
MTHFD_reverse_c10fd,-0.515687,0.000000,Folate Metabolism
RPI_reverse_853a1,0.000000,1.578062,Pentose Phosphate Pathway


Let's see if our model is feasible now.

In [33]:
relaxed_solution = model.optimize()
relaxed_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.0
EX_cmp_e,0.000000,-0.0
EX_co2_e,20.755531,0.0
EX_cobalt2_e,-0.000017,-0.0
DM_4crsol_c,0.000156,0.0
...,...,...
RNDR4,0.000000,0.0
RNDR4b,0.000000,0.0
RNTR1c2,0.018316,0.0
RNTR2c2,0.018912,0.0


In [34]:
relaxed_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7

And now we're also at the point where we can let the power of constraint based models work for us and make use of the tools mentioned above in order to adjust the fluxes calculated using MFA so that they nicely fit into our model.

In [35]:
relaxed_sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp1ecfzv6i.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpvi3v_3yb.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [36]:
relaxed_fluxes_sampling = reshape_fluxes_escher(relaxed_sampled_fluxes)

In [37]:
relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = relaxed_fluxes_sampling).display_in_notebook()
relaxed_flux_map

And as before, we can use alternative optimization algorithms here, like MOMA or ROOM.

In [38]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpty96mpwk.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpeaao333y.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp7qcieayz.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpn7w_gu2i.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding DAPDC
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp_mk73gr4.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpd6g857dp.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding BIOMASS_Ec_iJO1366_core_53p95M
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp8iec19zk.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpa1y_t4mq.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding GLNS
Read LP format model from file /var/folders

In [39]:
pfba_relaxed_solution = cobra.flux_analysis.pfba(model)

In [40]:
moma_after_relaxed_MFA = cobra.flux_analysis.moma(
    model=model, solution=pfba_relaxed_solution, linear=True)

In [41]:
moma_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745775968

In [42]:
room_after_relaxed_MFA = cobra.flux_analysis.room(
    model=model, solution=pfba_relaxed_solution, linear=True, delta=0.03, epsilon=0.001)

In [43]:
room_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7430984745776016

In [44]:
from tabulate import tabulate
results = [['Before adding new constraints:', round(original_solution.objective_value, 2)],
           ['New constraints relaxed FBA:', round(relaxed_solution.objective_value, 2)],
           ['New constraints relaxed pFBA:', round(pfba_relaxed_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed MOMA:', round(moma_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)],
           ['New constraints relaxed ROOM:', round(room_after_relaxed_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"], 2)]]
print(tabulate(results, headers=["Method", "Biomass function"]))

Method                            Biomass function
------------------------------  ------------------
Before adding new constraints:                0.98
New constraints relaxed FBA:                  0.7
New constraints relaxed pFBA:                 0.74
New constraints relaxed MOMA:                 0.74
New constraints relaxed ROOM:                 0.74


In [45]:
fluxes_relaxed_moma = reshape_fluxes_escher(moma_after_relaxed_MFA)
moma_relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                       reaction_data = fluxes_relaxed_moma).display_in_notebook()
moma_relaxed_flux_map

In [46]:
fluxes_relaxed_room = reshape_fluxes_escher(room_after_relaxed_MFA)
room_relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                       reaction_data = fluxes_relaxed_room).display_in_notebook()
room_relaxed_flux_map